Some times more than generic searches, we would like to exploit the fact that we count with a trustful resource, a curated knowledge that the agent can leverage.

In [1]:
from dotenv import load_dotenv

load_dotenv(override=True)

True

We will create a knowledge base the agent can retrieve information from and add it to the context of the LLM before solving any question. A critical piece is the **embedder**.

An embedder converts complex information into vector representations, allowing it to be stored in a vector database. By transforming data into embeddings, the embedder enables efficient searching and retrieval of contextually relevant information. This process enhances the responses of language models by providing them with the necessary business context, ensuring they are context-aware.

#### Vector search

We might need to add a Vector database to our system in order to allow for such a knowledge search as the embeddings of the chunks of text extracted from the knowledge base need to be transformed into the architecture for the agent to perform its search. [FAISS](https://faiss.ai/) is a well known system for vector storage and retrieval, originally created at Facebook (now Meta). Pretty basic in comparison with fully-fledged cloud hosted vector databases but it might well do the trick.

Let's first select a site to load data from: https://python.langchain.com/docs/introduction/

In [2]:
from langchain.document_loaders import WebBaseLoader

loader = WebBaseLoader("https://python.langchain.com/docs/introduction/")
documents = loader.load()

USER_AGENT environment variable not set, consider setting it to identify your requests.


Now, we need to create an embedding mechanism so that it stores all the information in a way our agent can search.

In [3]:
from langchain.vectorstores import FAISS
from langchain_google_genai import GoogleGenerativeAIEmbeddings

embedding_model = GoogleGenerativeAIEmbeddings(model="models/embedding-001")
vectorstore = FAISS.from_documents(documents, embedding_model)

Langchain already provides a nice Question Answer chain with Retrieval mechanism that asks to the knowledge DB before the final prompt is sent to the LLM.

In [4]:
from langchain.chains import RetrievalQA
from langchain_google_genai import ChatGoogleGenerativeAI

llm = ChatGoogleGenerativeAI(
    model="gemini-2.5-flash", 
    temperature=0,
    max_tokens=None,
    timeout=None,
    max_retries=2,
)

# Our chain
qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    retriever=vectorstore.as_retriever(),
    return_source_documents=True
)

Then we can simply ask.

In [5]:
result = qa_chain.invoke({"query": "What do you know about LangChain?"})
print(result['result'])

LangChain is a framework designed for developing applications powered by large language models (LLMs).

Here's what I know about LangChain:

*   **Purpose:** It simplifies every stage of the LLM application lifecycle, from development to productionization and deployment.
*   **Development:** It provides open-source components and integrates with third-party tools. It also uses LangGraph for building stateful agents with streaming and human-in-the-loop support.
*   **Productionization:** It integrates with LangSmith for inspecting, monitoring, and evaluating applications.
*   **Deployment:** It allows turning LangGraph applications into production-ready APIs and Assistants via LangGraph Platform.
*   **Standardization & Integrations:** LangChain implements a standard interface for LLMs, embedding models, and vector stores, and integrates with hundreds of providers.
*   **Architecture:** The framework consists of several open-source libraries:
    *   `langchain-core`: Base abstractions 

We can check the source documents it has used to augment the context to our prompt.

In [6]:
print(result["source_documents"])

[Document(id='9db2da45-a91e-4062-b4a5-f5e947dd7b3f', metadata={'source': 'https://python.langchain.com/docs/introduction/', 'title': 'Introduction | \uf8ffü¶úÔ∏è\uf8ffüîó LangChain', 'description': 'LangChain is a framework for developing applications powered by large language models (LLMs).', 'language': 'en'}, page_content='\n\n\n\n\nIntroduction | \uf8ffü¶úÔ∏è\uf8ffüîó LangChain\n\n\n\n\n\n\n\n\nSkip to main contentOur Building Ambient Agents with LangGraph course is now available on LangChain Academy!IntegrationsAPI ReferenceMoreContributingPeopleError referenceLangSmithLangGraphLangChain HubLangChain JS/TSv0.3v0.3v0.2v0.1\uf8ffüí¨SearchIntroductionTutorialsBuild a Question Answering application over a Graph DatabaseTutorialsBuild a simple LLM application with chat models and prompt templatesBuild a ChatbotBuild a Retrieval Augmented Generation (RAG) App: Part 2Build an Extraction ChainBuild an AgentTaggingBuild a Retrieval Augmented Generation (RAG) App: Part 1Build a semantic sea

And also check the traces in LangSmith.

![rag](../images/rag_faiss.png)